In [1]:
import pandas as pd
import numpy as np
import random as rand

In [18]:
df = pd.read_csv('jester-data-1.csv')
df_copy = df.copy()

In [19]:
n_users = df_copy.shape[0]
n_jokes = df_copy.shape[1]
n_total = n_users * n_jokes

n_train = n_total - (df_copy.iloc[:,:] == 99).sum().sum()
n_to_change = round(n_train * 0.1).astype(int)

(row, col) = (0,0)
for x in range(0, n_to_change):  
    while (df_copy.iloc[row, col]) == 99:
        row = rand.randint(0, users - 1)
        col = rand.randint(0, jokes - 1)
    df_copy.at[row, col] = 99    

df_copy.head()

,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,...,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25,0
0,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07,99.0
1,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,...,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,NaN
2,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,...,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00,NaN
3,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60,NaN
4,100,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,...,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45,NaN


In [20]:
latent_item_features = np.random.random((n_jokes, 3))
latent_user_preferences = np.random.random((n_users, 3))

In [ ]:
# Calculate the performance of the algorithm on the validation dataset
def predict_rating_from(latent_user_pref, latent_item_features):
    def predict_rating_(user_idx, item_idx):
        """ Predict a rating given a user_row and an item_row. """
    
        user_values = latent_user_pref[user_idx]
        item_values = latent_item_features[item_idx]
        return user_values.dot(item_values)
    return predict_rating_

# Training function.
def train(latent_user_pref, latent_item_features):
    predict_rating = predict_rating_from(latent_user_pref, latent_item_features)
    def train_(user_idx, item_idx, rating, alpha = 0.001):
        """ Adapt the values of user_preferences and item_factors to match
        the ones predicted by the users. """
    
        err = alpha * (rating - predict_rating(user_idx, item_idx))
        latent_user_pref[user_idx] += err * latent_item_features[item_idx]
        latent_item_features[item_idx] += err * latent_user_pref[user_idx]
        return err
    return train_


def sgd(iterations = 1000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    train_on_data = train(latent_user_preferences, latent_item_features)
    for iteration in range(0,iterations):
        training_error = []
        for user_idx in range(0, 100):
            for joke_idx in range(0, n_jokes):
                rating = df_copy.values[user_idx][joke_idx]
                if(not np.isnan(rating) and rating < 99):
                    training_error.append(train_on_data(user_idx, joke_idx, rating))
    mse = (np.array(training_error) ** 2).mean()          
    print (mse)

In [ ]:
sgd()

In [ ]:
#MSE
validation_error = []
for user in range(0, 100):
    for joke in range(0, jokes):
        rating_training_data = df.iloc[user][joke]
        rating_validation_data = df_copy.iloc[user][joke]
        if (not rating_training_data == rating_validation_data):
            prediction = predict_rating(user, joke)
            validation_error.append(rating_training_data - predict_rating(user, joke))
            #print("Expected: " + str(rating_training_data) + "\tActual: " + str(predict_rating(user, joke)))
print("Validation MSE: " + str((np.array(validation_error) ** 2).mean()))